<a href="https://colab.research.google.com/github/ariG23498/examples/blob/ariG23498%2Fsample-log/colabs/wandb-log/Performant_Sampled_Logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

<div><img /></div>

<img src="https://i.imgur.com/uEtWSEb.png" width="650" alt="Weights & Biases" />

<div><img /></div>

# 🪵 Too many logs spoil the perf(ormance)

Logging with [W&B](wandb.ai) is both easy and efficient. Someone new to wandb can find [tutorials on logging](https://github.com/wandb/examples/tree/master/colabs/wandb-log) quite helpful to start with. 

There are instances when there is a need to log metrics **frequently**. This is the case where we log:
- loss on _every batch_ 
- reward on _each step_ of each episode
- gradients on _every batch_

This can lead to substantial slowdown if the logging calls are much slower than the training iterations -- training will finish, but `wandb` will still be catching up.

In this notebook we will show you how to improve the performance of `wandb.log()` in such cases by:
- Sub-sampling
- Summarizing
- Batching

# Setup
To get started, just pip install the package and login from the command line.

If this is your first time using wandb, you'll need to sign up. It's easy!

In [1]:
%%capture
# install the package
! pip install wandb -qq

In [2]:
# login with the credentials
! wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import wandb
import random

timing_header = "=" * 5 + " Timing Results " + "=" * 5

# 🧲 Sub-sampling

In the code below we will _sample_ iterations and log the metrics corresponding to it. This way we will be logging at specific intervals in which `wandb` is not burdened with a lot of consecutive `wandb.log` calls.

In order to obtain the correct visualization one ought to log the correct number of sampled iteration too. This goes to the x-axis while we pull up charts for the logged metrics.

In [4]:
%%time

total_steps = 100_000
log_every = 500

run = wandb.init(project="perf-log",
                 name="sub-sampling")
with run:
    for iter in range(total_steps):
        metric = random.randint(1, 100)

        if (iter + 1) % log_every == 0:
            run.log({"metric": metric, "sample": iter})

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)

wandb: Currently logged in as: arig23498 (use `wandb login --relogin` to force relogin)


Run Finished!


metric,39
sample,99999
_runtime,3
_timestamp,1618470853
_step,199


metric,▂▄▂▂▄▃▇▁▇▅█▃█▇▇▁▆▆▂▇█▁▁▄▇▆▂▂▂▆▃▄▃▂▇▄▅▅█▄
sample,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Logging Finished!

===== Timing Results =====
CPU times: user 525 ms, sys: 36.2 ms, total: 562 ms
Wall time: 8.15 s


# 📝 Summarizing

This is a simple tweak to the `sub-sampling` method proposed above. Here we will be running a mean on the metrics and logging it on specific sampled intervals.

In [6]:
%%time

total_steps = 100_000
log_every = 500
metric = []

run = wandb.init(project="perf-log",
                 name="summarizing")

with run:
    for iter in range(total_steps):
        metric.append(random.randint(1, 100))

        if (iter + 1) % log_every == 0:
            mean_metric = sum(metric)//log_every
            run.log({"metric":mean_metric, "sample": iter})
            metric = []

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)

Run Finished!


metric,51
sample,99999
_runtime,3
_timestamp,1618472097
_step,199


metric,▄▇▇▂▂▅▇▄▄█▅▇▄█▅▇▇▁█▂▇▂▅▅▅▇▅▇▅▄▇▇▇▇▇▇█▇▇▇
sample,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Logging Finished!

===== Timing Results =====
CPU times: user 468 ms, sys: 33.1 ms, total: 501 ms
Wall time: 6.26 s


# 🏘️ Batching

In the code below we will be logging batches of data at regular intervals. With this approach we are essentially logging each and every metric unlike the approaches specified above.

In this method `wandb` builds a histogram of the batched metrics. This way one can gain insights about the statistical importance of the metrics.

In [7]:
%%time

total_steps = 100_000
log_every = 500
metric = []

run = wandb.init(project="perf-log",
                 name="batch")
with run:

    for iter in range(total_steps):
        metric.append(random.randint(1, 100))

        if (iter + 1) % log_every == 0:
            run.log({"metric": metric})
            metric = []

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)

Run Finished!


_runtime,3
_timestamp,1618472780
_step,199


_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Logging Finished!

===== Timing Results =====
CPU times: user 501 ms, sys: 36.5 ms, total: 537 ms
Wall time: 6.07 s


# 🐌Log everything

> In this section we log metrics for each iteration. This is to show how time consuming this process can be. <font color="red">Warning!</font> This cell takes a few minutes to run.

In [8]:
%%time

total_steps = 100_000
log_every = 500

run = wandb.init(project="perf-log",
                 name="log-everything") 
with run:

    for iter in range(total_steps):
        metric = random.randint(1, 100)
        run.log({"metric": metric})

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)

Run Finished!


metric,98
_runtime,19
_timestamp,1618473128
_step,99999


metric,▅▁▂▄█▆▆▇▇▆▁▆▅▁▂▁▅▃▁█▃▆▆▇▆▄▇▆▆▃▆▄▇▇▇▃▅▂▆▁
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Logging Finished!

===== Timing Results =====
CPU times: user 19.1 s, sys: 896 ms, total: 20 s
Wall time: 2min 10s
